<a href="https://colab.research.google.com/github/zlximon/Tensorflow_DL/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
#downloading helper functions 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2022-06-26 16:14:18--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-26 16:14:18 (72.5 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [60]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys



#get text dataset

In [61]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-06-26 16:14:18--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-06-26 16:14:18 (156 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



  #visualizing text dataset

In [62]:
import pandas as pd 

In [63]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()
# 1 is for disaster 

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [64]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [65]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [66]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [67]:
len(train_df),len(test_df)

(7613, 3263)

In [68]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Investigators shift focus to cause of fatal Waimate fire http://t.co/GoeTJGIhOp

---

Target: 0 (not real disaster)
Text:
First Time Playing Hearthstone on PC Thoughts http://t.co/aBoLxMH1vy

---

Target: 1 (real disaster)
Text:
Damage to school bus on 80 in multi car crash #BREAKING 

---

Target: 1 (real disaster)
Text:
@PrablematicLA @Adweek I'm actually currently dressed for a snowstorm...despite being in the middle of a Texas summer. Thanks office A/C.

---

Target: 1 (real disaster)
Text:
I liked a @YouTube video http://t.co/z8Cp77lVza Boeing 737 takeoff in snowstorm. HD cockpit view + ATC audio - Episode 18

---



#split data into training and validation  sets

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),train_df_shuffled["target"],test_size=0.1,
                                                                       random_state=42)

In [71]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [72]:
train_sentences[:10],train_labels[:10 ]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), 5921    0


Embedding : create a matrix of features vector for each token 

#text vectorization ( tokenization)

In [73]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [74]:
text_vectorizer=TextVectorization(max_tokens=None# how many word in vocalubary  and None means there is no cap 
                                  ,standardize="lower_and_strip_punctuation",# turn all words to lower case and strip 
                                  split="whitespace",ngrams=None,output_mode="int",output_sequence_length=None,# how long do we want sequences be 
                                 )# pad_to_max_tokens=True)#add zeros into end of short lenght 

In [75]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None # how long do you want your sequences to be?
                                    )

In [76]:
#find the average numbers of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [77]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [78]:
# fit the text vectorization to the training text
text_vectorizer.adapt(train_sentences)


In [79]:
#create a sample 
sample="There is a flood in my street"
text_vectorizer([sample])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [80]:
#choose a random sentence from training dataset
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Check this out! Lightning reshapes rocks at the atomic level http://t.co/l1gH8064YV #scichat #science        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 256,   19,   36,  300, 4751, 1999,   17,    2,  210,  813,    1,
        8754, 1770,    0,    0]])>

In [81]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating Embedding layer 

Embedding : turns positives integers into dense vectors of fixed size
parameters for our embedding

1-input_dim - The size of the vocabulary

2-output_dim - The size of the output embedding vector, for example, a value of 100 outputs a feature vector of size 100 for each word.

3-embeddings_initializer - How to initialize the embeddings matrix, default is "uniform" which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.

4- input_length - Length of sequences being passed to embedding layer.


In [82]:
from tensorflow.keras import layers 
embedding =layers.Embedding(input_dim=max_vocab_length,output_dim=128,input_length=max_length)
embedding

In [83]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Freak accident? Sure. Looking for someone to blame? Maybe. Remember that player broke his leg cuz cart was at back of end zone? Common sense      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03400532, -0.00257919, -0.04307525, ..., -0.00209256,
         -0.02727911,  0.03539443],
        [ 0.01983817, -0.04703108, -0.01306349, ...,  0.03981983,
         -0.00109838,  0.00243949],
        [-0.03576244, -0.02310681,  0.04548686, ...,  0.02144219,
         -0.04485387,  0.04427189],
        ...,
        [ 0.0386024 ,  0.02714405, -0.01278319, ...,  0.01946248,
          0.0197502 , -0.02532355],
        [ 0.00213682,  0.02554277, -0.04735595, ...,  0.02053659,
         -0.0212608 , -0.02523447],
        [ 0.03543197, -0.01656847,  0.02876897, ...,  0.01874384,
         -0.00284274,  0.04668472]]], dtype=float32)>

In [84]:
sample_embed[0][0],sample_embed[0][0].shape,random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03400532, -0.00257919, -0.04307525, -0.03912424, -0.04738137,
         0.03095442,  0.0217698 ,  0.02441961,  0.04181372, -0.03446944,
        -0.00234344, -0.03512956, -0.02899429, -0.02501276,  0.04408539,
        -0.04075759,  0.01546264, -0.02337298, -0.04865589, -0.0463929 ,
         0.04050993, -0.00873965,  0.03752167, -0.02664613, -0.03123829,
        -0.006743  ,  0.00582751, -0.03906015, -0.04294472,  0.04739625,
         0.04187271, -0.02661963,  0.04063919, -0.03685429,  0.02010392,
        -0.00194464,  0.00020001,  0.04745361, -0.00748206,  0.01626882,
         0.01913441,  0.03016337, -0.04080179, -0.04259505,  0.00849883,
        -0.02670358, -0.04540359, -0.00219589,  0.04237868, -0.012201  ,
         0.02762102,  0.04482624,  0.03599692, -0.00902418,  0.02172288,
         0.00192352,  0.02070773, -0.0103941 , -0.02257034,  0.00658293,
        -0.02513399, -0.01370997,  0.00793809,  0.04231223, -0.03862493,
  

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [86]:
model0=Pipeline([
("tfidf",TfidfVectorizer()),#convert words to numbers using tfdif
("clf",MultinomialNB())#model the text
])
#fit pipeline to train data
model0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [87]:
baseline_score=model0.score(val_sentences,val_labels)# evaluate 
baseline_score # accuracy 

0.7926509186351706

In [88]:
baseline_pred=model0.predict(val_sentences)
baseline_pred[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

#creating function to track and evaluate model 

In [89]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [90]:
baseline_result=calculate_results(y_true=val_labels,y_pred=baseline_pred)
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#model 1 (Dense model)

In [91]:
from helper_functions import create_tensorboard_callback


In [92]:
SAVE_DIR = "model_logs"


In [93]:
from tensorflow.keras import layers
inputs=(layers.Input(shape=(1,),dtype=tf.string))
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
#x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model1=tf.keras.Model(inputs,outputs,name="model1")

In [94]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [95]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [96]:
#compile model
model1.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [97]:
#fit the model
model1_history=model1.fit(x=train_sentences,y=train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220626-161420
Epoch 1/5
215/215 [==============================] - 2s 6ms/step - loss: 0.6126 - accuracy: 0.6903 - val_loss: 0.5363 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4417 - accuracy: 0.8193 - val_loss: 0.4717 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3464 - accuracy: 0.8594 - val_loss: 0.4581 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2838 - accuracy: 0.8917 - val_loss: 0.4626 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2379 - accuracy: 0.9110 - val_loss: 0.4748 - val_accuracy: 0.7835


In [98]:
model1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4748 - accuracy: 0.7835


[0.47484415769577026, 0.7834645509719849]

In [99]:
model1_pred_probs=model1.predict(val_sentences)
model1_pred_probs.shape

(762, 1)

In [100]:
model1_pred_probs[0]

array([0.44206977], dtype=float32)

In [101]:
#Convert model prediction probabilites to label format 
model1_preds=tf.squeeze(tf.round(model1_pred_probs))
model1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [102]:
model1_result=calculate_results(y_true=val_labels,y_pred=model1_preds)
model1_result


{'accuracy': 78.34645669291339,
 'f1': 0.7811856084066165,
 'precision': 0.7861644127328657,
 'recall': 0.7834645669291339}

In [103]:
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#visualazing learned embeddings

In [104]:
#get the vocabulary from  the vectorization layer
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [105]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [107]:
embed_weights=model1.get_layer("embedding").get_weights()[0]
embed_weights.shape

ValueError: ignored

In [108]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [109]:
# Let's download the saved embeddings locally

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>